In [22]:
import requests
from sseclient import SSEClient
from pprint import pprint

# Base URL of your FastAPI server
BASE_URL = "http://127.0.0.1:8765"

# Function to get all threads
def get_threads():
    response = requests.get(f"{BASE_URL}/threads")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Function to get a specific thread
def get_thread(thread_id):
    response = requests.get(f"{BASE_URL}/thread/{thread_id}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def chat(query, thread_id=None, model=None, temperature=None):
    data = {
        "query": query,
        "thread_id": thread_id,
        "model": model,
        "temperature": temperature
    }
    response = requests.post(f"{BASE_URL}/chat", json=data, stream=True)
    
    if response.status_code == 200:
        final_output = ""
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith('data: '):
                    event_data = decoded_line[6:]
                    if event_data == '[DONE]':
                        break
                    print(event_data, end='', flush=True)
                    final_output += event_data
        print()  # Add a newline at the end of the response
        return final_output
    else:
        print(f"Error: {response.status_code}")
        return None

In [17]:
print("Getting all threads:")
threads = get_threads()
pprint(threads)
print()

Getting all threads:
[{'thread_id': '038bdbe8-83d6-4b8e-8aba-9cf11de511dd',
  'thread_name': 'Client Meeting with User Feedback and Discussion',
  'timestamp': '2024-10-17T18:40:30.160965'},
 {'thread_id': '041c54e1-54da-4ee3-8d91-4509156d01e2',
  'thread_name': 'Assessing Potential Vexa Customers Among Speakers',
  'timestamp': '2024-10-18T12:52:44.042996'},
 {'thread_id': '083f9925-9c74-4ef1-929a-f5e81b73f92c',
  'thread_name': 'Identifying Vexa Customers Among Speakers',
  'timestamp': '2024-10-18T13:56:48.396266'},
 {'thread_id': '09d93006-108a-4f0a-988d-708661f87026',
  'thread_name': 'Communications with Matt Lewis',
  'timestamp': '2024-10-17T18:00:57.482037'},
 {'thread_id': '0dff7138-83eb-433b-b546-2d4a3a999702',
  'thread_name': 'Identifying Vexa Customers Among Speakers',
  'timestamp': '2024-10-18T14:07:13.837322'},
 {'thread_id': '0e02a54a-b4f2-4aa8-b62b-aaaa9a0d07a1',
  'thread_name': 'Identifying Vexa Customers Among Unique Speakers',
  'timestamp': '2024-10-18T12:49:22.

In [18]:

# Get a specific thread (assuming we have a thread_id)
if threads:
    thread_id = threads[0]['thread_id']
    print(f"Getting thread {thread_id}:")
    thread = get_thread(thread_id)
    pprint(thread)
    print()

Getting thread 038bdbe8-83d6-4b8e-8aba-9cf11de511dd:
{'messages': [{'content': 'identify all meetings with users and clients, '
                          'discurd meetings with coworkers',
               'role': 'user',
               'stage': None},
              {'content': 'The meetings that involved users and clients are as '
                          'follows:\n'
                          '\n'
                          '1. **Technical Troubleshooting Meeting** '
                          '(2024-08-30 18:01): This meeting involved '
                          'discussions related to user registrations and '
                          'website settings, indicating user interaction[2].\n'
                          '\n'
                          '2. **AI Tools in Product Development Discussion** '
                          '(2024-09-04 18:00): This meeting included '
                          'discussions on the use of AI tools and user '
                          'experiences, suggesti

In [21]:

# Start a new chat session
print("Starting a new chat session:")
chat("Tell me about the latest marketing efforts")
print()

Starting a new chat session:
The latest marketing efforts discussed by Dmitriy Grankin and Olga Nemirovskaya include a variety of strategies focused on enhancing user engagement, improving conversion rates, and leveraging different platforms for outreach.1. **Google Ads and Performance Tracking**: They reviewed the performance of Google Ads campaigns, noting the importance of targeting English-speaking countries and optimizing user engagement metrics. A focus was placed on refining their strategy through iterative testing and effective tracking of conversion statistics[4][14].2. **Content Strategy on Social Media**: They emphasized the need for a robust content strategy across various platforms, particularly LinkedIn and TikTok. This involves creating diverse types of content, including light entertainment posts, informative content, and sales-driven messaging. They formulated plans for increasing post frequency to boost visibility and user engagement[1][3].3. **Influencer Collaboratio

In [23]:
query = 'summarize our marketing efforts'

In [24]:
final_result = chat(query)
if final_result:
    print("Final output:", final_result)

The marketing efforts discussed among Dmitriy Grankin and Olga Nemirovskaya include a multifaceted approach that concentrates on user engagement, content strategies, and leveraging various digital platforms. Key highlights are:1. **Mobile Advertising and Influencer Collaborations**: The team focused on mobile advertising and collaborating with influencers on platforms like LinkedIn and TikTok to enhance visibility and drive engagement. They emphasized the importance of testing new strategies and analyzing performance metrics to refine their efforts[1][2].2. **Content Strategy and Post Frequency**: A clear content strategy was discussed, aiming for a mix of entertaining, educational, and promotional posts. The team considered increasing the frequency of posts on LinkedIn to boost engagement, believing that more regular touches with content would enhance visibility and connection with users[1][3].3. **Analytics and User Feedback**: They identified the need for robust analytics to track u

In [26]:
final_result


'The marketing efforts discussed among Dmitriy Grankin and Olga Nemirovskaya include a multifaceted approach that concentrates on user engagement, content strategies, and leveraging various digital platforms. Key highlights are:1. **Mobile Advertising and Influencer Collaborations**: The team focused on mobile advertising and collaborating with influencers on platforms like LinkedIn and TikTok to enhance visibility and drive engagement. They emphasized the importance of testing new strategies and analyzing performance metrics to refine their efforts[1][2].2. **Content Strategy and Post Frequency**: A clear content strategy was discussed, aiming for a mix of entertaining, educational, and promotional posts. The team considered increasing the frequency of posts on LinkedIn to boost engagement, believing that more regular touches with content would enhance visibility and connection with users[1][3].3. **Analytics and User Feedback**: They identified the need for robust analytics to track 